In [52]:
# Importing dependencies
import pandas as pd
from splinter import Browser
import pymongo
import json
import csv

# Test Data

In [2]:
# Displaying test and gender csv
test_df = pd.read_csv('../Resources/test.csv')
gender_submission_df = pd.read_csv('../Resources/gender_submission.csv')

display(test_df)
display(gender_submission_df)

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...
413,1305,3,"Spector, Mr. Woolf",male,NaN,0,0,A.5. 3236,8.0500,NaN,S
414,1306,1,"Oliva y Ocana, Dona. Fermina",female,39.0,0,0,PC 17758,108.9000,C105,C
415,1307,3,"Saether, Mr. Simon Sivertsen",male,38.5,0,0,SOTON/O.Q. 3101262,7.2500,NaN,S
416,1308,3,"Ware, Mr. Frederick",male,NaN,0,0,359309,8.0500,NaN,S


,PassengerId,Survived
0,892,0
1,893,1
2,894,0
3,895,0
4,896,1
...,...,...
413,1305,0
414,1306,1
415,1307,0
416,1308,0


In [3]:
# Checking for any nulls
display(test_df.info())
display(gender_submission_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  418 non-null    int64  
 1   Pclass       418 non-null    int64  
 2   Name         418 non-null    object 
 3   Sex          418 non-null    object 
 4   Age          332 non-null    float64
 5   SibSp        418 non-null    int64  
 6   Parch        418 non-null    int64  
 7   Ticket       418 non-null    object 
 8   Fare         417 non-null    float64
 9   Cabin        91 non-null     object 
 10  Embarked     418 non-null    object 
dtypes: float64(2), int64(4), object(5)
memory usage: 36.1+ KB


None

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype
---  ------       --------------  -----
 0   PassengerId  418 non-null    int64
 1   Survived     418 non-null    int64
dtypes: int64(2)
memory usage: 6.7 KB


None

In [4]:
# Merging gender_df into test_df
test_df_merged = test_df.merge(gender_submission_df, on='PassengerId', how='left')
test_df_merged


,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Survived
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q,0
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S,1
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q,0
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S,0
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S,1
...,...,...,...,...,...,...,...,...,...,...,...,...
413,1305,3,"Spector, Mr. Woolf",male,NaN,0,0,A.5. 3236,8.0500,NaN,S,0
414,1306,1,"Oliva y Ocana, Dona. Fermina",female,39.0,0,0,PC 17758,108.9000,C105,C,1
415,1307,3,"Saether, Mr. Simon Sivertsen",male,38.5,0,0,SOTON/O.Q. 3101262,7.2500,NaN,S,0
416,1308,3,"Ware, Mr. Frederick",male,NaN,0,0,359309,8.0500,NaN,S,0


In [5]:
# Splitting Name to first and last
test_df_merged[['last_name', 'first_name']] = test_df_merged['Name'].str.split(', ', expand=True)
# Removing period in first name to make matching easier later
test_df_merged['first_name'] = test_df_merged['first_name'].str.replace('.', '')
# Removing extra quotes some first names have
test_df_merged['first_name'] = test_df_merged['first_name'].str.replace('"', '')
# Reorganizing columns and dropping Cabin/Name column
test_df_dropped = test_df_merged[['PassengerId', 'Survived', 'Pclass', 'last_name','first_name', 'Sex', 'Age', 'SibSp', 'Parch',
       'Ticket', 'Fare', 'Embarked']]

test_df_dropped

,PassengerId,Survived,Pclass,last_name,first_name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked
0,892,0,3,Kelly,Mr James,male,34.5,0,0,330911,7.8292,Q
1,893,1,3,Wilkes,Mrs James (Ellen Needs),female,47.0,1,0,363272,7.0000,S
2,894,0,2,Myles,Mr Thomas Francis,male,62.0,0,0,240276,9.6875,Q
3,895,0,3,Wirz,Mr Albert,male,27.0,0,0,315154,8.6625,S
4,896,1,3,Hirvonen,Mrs Alexander (Helga E Lindqvist),female,22.0,1,1,3101298,12.2875,S
...,...,...,...,...,...,...,...,...,...,...,...,...
413,1305,0,3,Spector,Mr Woolf,male,NaN,0,0,A.5. 3236,8.0500,S
414,1306,1,1,Oliva y Ocana,Dona Fermina,female,39.0,0,0,PC 17758,108.9000,C
415,1307,0,3,Saether,Mr Simon Sivertsen,male,38.5,0,0,SOTON/O.Q. 3101262,7.2500,S
416,1308,0,3,Ware,Mr Frederick,male,NaN,0,0,359309,8.0500,S


In [6]:
# Creating Deck column, binning passengers by their class
deck_floors_test = []


for class_status in test_df_dropped['Pclass']:
    if class_status == 1:
        deck_floors_test.append('A,B,C,D,E')
    else:
        deck_floors_test.append('D,E,F,G')

i = 0

for k in range(len(test_df_dropped)):
    test_df_dropped.loc[k, ['Deck']] = deck_floors_test[i]
    i += 1


test_df_dropped

,PassengerId,Survived,Pclass,last_name,first_name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked,Deck
0,892,0,3,Kelly,Mr James,male,34.5,0,0,330911,7.8292,Q,"D,E,F,G"
1,893,1,3,Wilkes,Mrs James (Ellen Needs),female,47.0,1,0,363272,7.0000,S,"D,E,F,G"
2,894,0,2,Myles,Mr Thomas Francis,male,62.0,0,0,240276,9.6875,Q,"D,E,F,G"
3,895,0,3,Wirz,Mr Albert,male,27.0,0,0,315154,8.6625,S,"D,E,F,G"
4,896,1,3,Hirvonen,Mrs Alexander (Helga E Lindqvist),female,22.0,1,1,3101298,12.2875,S,"D,E,F,G"
...,...,...,...,...,...,...,...,...,...,...,...,...,...
413,1305,0,3,Spector,Mr Woolf,male,NaN,0,0,A.5. 3236,8.0500,S,"D,E,F,G"
414,1306,1,1,Oliva y Ocana,Dona Fermina,female,39.0,0,0,PC 17758,108.9000,C,"A,B,C,D,E"
415,1307,0,3,Saether,Mr Simon Sivertsen,male,38.5,0,0,SOTON/O.Q. 3101262,7.2500,S,"D,E,F,G"
416,1308,0,3,Ware,Mr Frederick,male,NaN,0,0,359309,8.0500,S,"D,E,F,G"


In [7]:
# Filling single null value in Fare column
fare_null_index = test_df_dropped.loc[test_df_dropped['Fare'].isna()].index
test_df_dropped.loc[fare_null_index, ['Fare']] = 7

In [8]:
# Checking age is only null column
test_df_dropped.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 13 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  418 non-null    int64  
 1   Survived     418 non-null    int64  
 2   Pclass       418 non-null    int64  
 3   last_name    418 non-null    object 
 4   first_name   418 non-null    object 
 5   Sex          418 non-null    object 
 6   Age          332 non-null    float64
 7   SibSp        418 non-null    int64  
 8   Parch        418 non-null    int64  
 9   Ticket       418 non-null    object 
 10  Fare         418 non-null    float64
 11  Embarked     418 non-null    object 
 12  Deck         418 non-null    object 
dtypes: float64(2), int64(5), object(6)
memory usage: 42.6+ KB


In [9]:
# Creating age nulls only df and extracting last_name, first_name, and id 
test_nulls_only_df = test_df_dropped.loc[test_df_dropped['Age'].isna()]

test_nulls_last_names = test_nulls_only_df['last_name'].values
test_nulls_first_names = test_nulls_only_df['first_name'].values
test_nulls_ids = test_nulls_only_df['PassengerId'].values

test_nulls_only_df

,PassengerId,Survived,Pclass,last_name,first_name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked,Deck
10,902,0,3,Ilieff,Mr Ylio,male,NaN,0,0,349220,7.8958,S,"D,E,F,G"
22,914,1,1,Flegenheim,Mrs Alfred (Antoinette),female,NaN,0,0,PC 17598,31.6833,S,"A,B,C,D,E"
29,921,0,3,Samaan,Mr Elias,male,NaN,2,0,2662,21.6792,C,"D,E,F,G"
33,925,1,3,Johnston,Mrs Andrew G (Elizabeth Lily Watson),female,NaN,1,2,W./C. 6607,23.4500,S,"D,E,F,G"
36,928,1,3,Roth,Miss Sarah A,female,NaN,0,0,342712,8.0500,S,"D,E,F,G"
...,...,...,...,...,...,...,...,...,...,...,...,...,...
408,1300,1,3,Riordan,Miss Johanna Hannah,female,NaN,0,0,334915,7.7208,Q,"D,E,F,G"
410,1302,1,3,Naughton,Miss Hannah,female,NaN,0,0,365237,7.7500,Q,"D,E,F,G"
413,1305,0,3,Spector,Mr Woolf,male,NaN,0,0,A.5. 3236,8.0500,S,"D,E,F,G"
416,1308,0,3,Ware,Mr Frederick,male,NaN,0,0,359309,8.0500,S,"D,E,F,G"


In [10]:
# Scraping data from titanic encyclopedia for unknown ages
browser = Browser('chrome')
url = 'https://titanicfacts.net/titanic-passenger-list/'
browser.visit(url)
html = browser.html
all_classes_tables = pd.read_html(html)

browser.quit()

In [11]:
# Writing each class table into a csv
i = 0
place = ['1st', '2nd', '3rd']

for table in all_classes_tables:
    table.to_csv(f'../Resources/{place[i]}_class.csv', header=False, index=False)
    i += 1

In [12]:
# Merging all csv's into one dataframe 
pd.concat(map(pd.read_csv, 
              ['../Resources/1st_class.csv', 
               '../Resources/2nd_class.csv', 
               '../Resources/3rd_class.csv']), 
               ignore_index=True).to_csv('../Resources/merged_online_table.csv', index=False)


In [13]:
# Viewing newly made single dataframe 
full_table_df  = pd.read_csv('../Resources/full_online_table')
full_table_df

,Surname,First Names,Age,Boarded,Survivor (S) or Victim (†)
0,Allen,Miss Elisabeth Walton,29,Southampton,S
1,Allison,Mr Hudson Joshua Creighton,30,Southampton,†
2,Allison,Mrs Bessie Waldo,25,Southampton,†
3,Allison,Miss Helen Loraine,2,Southampton,†
4,Allison,Master Hudson Trevor,11m,Southampton,S
...,...,...,...,...,...
1312,Youssiff (Sam’Aan),Mr Gerios,45,Cherbourg,†
1313,Zakarian,Mr Ortin,27,Cherbourg,†
1314,Zakarian,Mr Mapriededer,22,Cherbourg,†
1315,Zenni,Mr Philip,22,Cherbourg,S


In [14]:
# Cleaning dataframe by renaming and removing columns
full_table_df = full_table_df[['Surname', 'First Names', 'Age']]
full_table_df = full_table_df.rename(columns={'Surname': 'last_name', 'First Names': 'first_name', 'Age': 'age'})

full_table_df

,last_name,first_name,age
0,Allen,Miss Elisabeth Walton,29
1,Allison,Mr Hudson Joshua Creighton,30
2,Allison,Mrs Bessie Waldo,25
3,Allison,Miss Helen Loraine,2
4,Allison,Master Hudson Trevor,11m
...,...,...,...
1312,Youssiff (Sam’Aan),Mr Gerios,45
1313,Zakarian,Mr Ortin,27
1314,Zakarian,Mr Mapriededer,22
1315,Zenni,Mr Philip,22


In [15]:
# Checking for nulls and data types
full_table_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1317 entries, 0 to 1316
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   last_name   1317 non-null   object
 1   first_name  1317 non-null   object
 2   age         1317 non-null   object
dtypes: object(3)
memory usage: 31.0+ KB


In [16]:
# Checking for any unusual values in age column to change data type
full_table_df['age'].unique()

array(['29', '30', '25', '2', '11m', '47', '62', '39', '53', '71', '18',
       '24', '26', '32', '60', '50', '36', '37', '46', '31', '19', '28',
       '45', '61', '42', '22', '41', '48', '59', '44', '58', '52', '33',
       '17', '27', '13', '11', '49', '43', '40', '64', '55', '70', '35',
       '51', '34', '4', '23', '54', '57', '38', '21', '16', '65', '20',
       '56', '6', '67', '63', '15', '12', '1', '10m', '8', '7m', '7', '3',
       '14', '9m', '9', '5', '66', '4m', '2m', 'NK', '10', '69', '74',
       '5m'], dtype=object)

In [17]:
# Viewing 'NK' (Not Known) rows
full_table_df.loc[full_table_df['age'] == 'NK']

,last_name,first_name,age
821,Gheorgheff,Mr Stanio,NK
948,Kraeff,Mr Theodor,NK


In [18]:
# Putting 'NK' indexes in a variable and storing their names to also remove in other dfs
nk_full_table_index = full_table_df.loc[full_table_df['age'] == 'NK'].index
nk_last_names = full_table_df.loc[full_table_df['age'] == 'NK']['last_name'].values
nk_first_names = full_table_df.loc[full_table_df['age'] == 'NK']['first_name'].values

In [19]:
# Dropping 'NK' rows and reseting index
full_table_df.drop(index=nk_full_table_index, inplace=True)
full_table_df.reset_index(inplace=True, drop=True)

full_table_df

,last_name,first_name,age
0,Allen,Miss Elisabeth Walton,29
1,Allison,Mr Hudson Joshua Creighton,30
2,Allison,Mrs Bessie Waldo,25
3,Allison,Miss Helen Loraine,2
4,Allison,Master Hudson Trevor,11m
...,...,...,...
1310,Youssiff (Sam’Aan),Mr Gerios,45
1311,Zakarian,Mr Ortin,27
1312,Zakarian,Mr Mapriededer,22
1313,Zenni,Mr Philip,22


In [20]:
# Replacing other string age values to match all float
full_table_df = full_table_df.replace({'11m': '0.9', 
                                    '10m': '0.8', 
                                    '9m': '0.7',
                                    '7m': '0.5',
                                    '5m': '0.4',
                                    '4m': '0.3', 
                                    '2m': '0.1'})


In [21]:
# Double checking
full_table_df['age'].unique()

array(['29', '30', '25', '2', '0.9', '47', '62', '39', '53', '71', '18',
       '24', '26', '32', '60', '50', '36', '37', '46', '31', '19', '28',
       '45', '61', '42', '22', '41', '48', '59', '44', '58', '52', '33',
       '17', '27', '13', '11', '49', '43', '40', '64', '55', '70', '35',
       '51', '34', '4', '23', '54', '57', '38', '21', '16', '65', '20',
       '56', '6', '67', '63', '15', '12', '1', '0.8', '8', '0.5', '7',
       '3', '14', '0.7', '9', '5', '66', '0.3', '0.1', '10', '69', '74',
       '0.4'], dtype=object)

In [22]:
# Changing age column data type to float to match other csv's
full_table_df_clean = full_table_df.astype({'age': float})

# # Viewing final clean dataframe
full_table_df_clean.info()

full_table_df_clean

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1315 entries, 0 to 1314
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   last_name   1315 non-null   object 
 1   first_name  1315 non-null   object 
 2   age         1315 non-null   float64
dtypes: float64(1), object(2)
memory usage: 30.9+ KB


,last_name,first_name,age
0,Allen,Miss Elisabeth Walton,29.0
1,Allison,Mr Hudson Joshua Creighton,30.0
2,Allison,Mrs Bessie Waldo,25.0
3,Allison,Miss Helen Loraine,2.0
4,Allison,Master Hudson Trevor,0.9
...,...,...,...
1310,Youssiff (Sam’Aan),Mr Gerios,45.0
1311,Zakarian,Mr Ortin,27.0
1312,Zakarian,Mr Mapriededer,22.0
1313,Zenni,Mr Philip,22.0


In [23]:
# Creating loop to find missing age values from full table to fill in for test csv
test_ages_to_fill = []
manually_fill_test_ages_empty = []

for index, last_name in enumerate(test_nulls_last_names):
    first_name = test_nulls_first_names[index]

    search = full_table_df_clean.loc[full_table_df_clean['last_name'] == last_name]   
    refined_search = search.loc[search['first_name'] == first_name]

    if len(refined_search) == 1:
        test_ages_to_fill.append(refined_search['age'].values[0])
    else:
        test_ages_to_fill.append(0)
        manually_fill_test_ages_empty.append({'last_name': last_name, 'first_name': first_name, 'age': ''})

In [24]:
# Printing results
print(test_ages_to_fill)
print(manually_fill_test_ages_empty)
print(f'We started with {len(test_ages_to_fill)} nulls')
print(f'Our search refined to manually go over {len(manually_fill_test_ages_empty)}')

[32.0, 0, 17.0, 0, 0, 24.0, 0, 24.0, 30.0, 32.0, 0, 23.0, 0, 0, 25.0, 0, 28.0, 23.0, 20.0, 24.0, 0, 0, 0, 0, 24.0, 0, 0, 0, 44.0, 59.0, 30.0, 17.0, 32.0, 0, 0, 20.0, 16.0, 0, 0, 0, 34.0, 29.0, 34.0, 20.0, 0, 0, 32.0, 21.0, 0, 0, 0, 0, 0, 44.0, 43.0, 30.0, 26.0, 28.0, 0, 0, 0, 0, 25.0, 22.0, 33.0, 0, 0, 0, 0, 0, 0, 0, 44.0, 0, 0, 0, 0, 28.0, 0, 0, 0, 0, 21.0, 23.0, 34.0, 0]
[{'last_name': 'Flegenheim', 'first_name': 'Mrs Alfred (Antoinette)', 'age': ''}, {'last_name': 'Johnston', 'first_name': 'Mrs Andrew G (Elizabeth Lily Watson)', 'age': ''}, {'last_name': 'Roth', 'first_name': 'Miss Sarah A', 'age': ''}, {'last_name': 'Franklin', 'first_name': 'Mr Thomas Parham', 'age': ''}, {'last_name': 'Corey', 'first_name': 'Mrs Percy C (Mary Phyllis Elizabeth Miller)', 'age': ''}, {'last_name': 'Demetri', 'first_name': 'Mr Marinko', 'age': ''}, {'last_name': 'Lamb', 'first_name': 'Mr John Joseph', 'age': ''}, {'last_name': "O'Donoghue", 'first_name': 'Ms Bridget', 'age': ''}, {'last_name': 'Will

In [25]:
# Creating list to fill ages
manually_fill_test_ages = [
 {'last_name': 'Flegenheim', 'first_name': 'Mrs Alfred (Antoinette)', 'age': 48},
 {'last_name': 'Johnston', 'first_name': 'Mrs Andrew G (Elizabeth Lily Watson)', 'age': 36},
 {'last_name': 'Roth', 'first_name': 'Miss Sarah A', 'age': 31},
 {'last_name': 'Franklin', 'first_name': 'Mr Thomas Parham', 'age': 37},
 {'last_name': 'Corey', 'first_name': 'Mrs Percy C (Mary Phyllis Elizabeth Miller)', 'age': 32},
 {'last_name': 'Demetri', 'first_name': 'Mr Marinko', 'age': 23},
 {'last_name': 'Lamb', 'first_name': 'Mr John Joseph', 'age': 30},
 {'last_name': "O'Donoghue", 'first_name': 'Ms Bridget', 'age': 21},
 {'last_name': 'Willer', 'first_name': 'Mr Aaron (Abi Weller)', 'age': 37},
 {'last_name': 'Shine', 'first_name': 'Miss Ellen Natalia', 'age': 20},
 {'last_name': 'Thomas', 'first_name': 'Mr John', 'age': 34},
 {'last_name': 'Kiernan', 'first_name': 'Mr John', 'age': 25},
 {'last_name': 'McCoy', 'first_name': 'Miss Alicia', 'age': 26},
 {'last_name': 'Lefebre', 'first_name': 'Mrs Frank (Frances)', 'age': 40},
 {'last_name': 'Thomas', 'first_name': 'Mr Charles P', 'age': 20},
 {'last_name': 'Cassebeer', 'first_name': 'Mrs Henry Arthur Jr (Eleanor Genevieve Fosdick)', 'age': 36},
 {'last_name': 'Lithman', 'first_name': 'Mr Simon', 'age': 27},
 {'last_name': 'Sage', 'first_name': 'Miss Ada', 'age': 10},
 {'last_name': 'Salomon', 'first_name': 'Mr Abraham L', 'age': 43},
 {'last_name': 'Rasmussen', 'first_name': 'Mrs (Lena Jacobsen Solvang)', 'age': 63},
 {'last_name': 'Thomson', 'first_name': 'Mr Alexander Morrison', 'age': 36},
 {'last_name': 'Moubarek', 'first_name': 'Mrs George (Omine Amenia Alexander)', 'age': 25},
 {'last_name': 'Hyman', 'first_name': 'Mr Abraham', 'age': 34},
 {'last_name': 'Johnston', 'first_name': 'Master William Arthur Willie', 'age': 8},
 {'last_name': 'Khalil', 'first_name': 'Mrs Betros (Zahie Maria Elias)', 'age': 20},
 {'last_name': 'MacKay', 'first_name': 'Mr George William', 'age': 20},
 {'last_name': 'Mahon', 'first_name': 'Mr John', 'age': 22},
 {'last_name': 'Lennon', 'first_name': 'Miss Mary', 'age': 18},
 {'last_name': 'Saade', 'first_name': 'Mr Jean Nassr', 'age': 20},
 {'last_name': 'Fleming', 'first_name': 'Miss Honora', 'age': 22},
 {'last_name': 'Franklin', 'first_name': 'Mr Charles (Charles Fardon)', 'age': 46},
 {'last_name': 'Nasr', 'first_name': 'Mr Mustafa', 'age': 20},
 {'last_name': 'Samaan', 'first_name': 'Mr Hanna', 'age': 40},
 {'last_name': 'Malachard', 'first_name': 'Mr Noel', 'age': 25},
 {'last_name': 'McCarthy', 'first_name': 'Miss Catherine Katie', 'age': 24},
 {'last_name': 'Sadowitz', 'first_name': 'Mr Harry', 'age': 17},
 {'last_name': 'Thomas', 'first_name': 'Mr Tannous', 'age': 16},
 {'last_name': 'Betros', 'first_name': 'Master Seman', 'age': 10},
 {'last_name': 'van Billiard', 'first_name': 'Master James William', 'age': 10},
 {'last_name': 'Lockyer', 'first_name': 'Mr Edward', 'age': 19},
 {'last_name': "O'Keefe", 'first_name': 'Mr Patrick', 'age': 21},
 {'last_name': 'Sage', 'first_name': 'Mrs John (Annie Bullen)', 'age': 44},
 {'last_name': "O'Connor", 'first_name': 'Mr Patrick', 'age': 23},
 {'last_name': 'Risien', 'first_name': 'Mrs Samuel (Emma)', 'age': 58},
 {'last_name': 'Wheeler', 'first_name': 'Mr Edwin Frederick', 'age': 26},
 {'last_name': 'Riordan', 'first_name': 'Miss Johanna Hannah', 'age': 21},
 {'last_name': 'Peter', 'first_name': 'Master Michael J', 'age': 4}
 ]

In [26]:
# Merging queried ages to make a complete list to fill test csv nulls
i = 0
for age in range(len(test_ages_to_fill)):
    if test_ages_to_fill[age] == 0:
       test_ages_to_fill[age] = manually_fill_test_ages[i]['age']
       i += 1

test_ages_to_fill

[32.0,
 48,
 17.0,
 36,
 31,
 24.0,
 37,
 24.0,
 30.0,
 32.0,
 32,
 23.0,
 23,
 30,
 25.0,
 21,
 28.0,
 23.0,
 20.0,
 24.0,
 37,
 20,
 34,
 25,
 24.0,
 26,
 40,
 20,
 44.0,
 59.0,
 30.0,
 17.0,
 32.0,
 36,
 27,
 20.0,
 16.0,
 10,
 43,
 63,
 34.0,
 29.0,
 34.0,
 20.0,
 36,
 25,
 32.0,
 21.0,
 34,
 8,
 20,
 20,
 22,
 44.0,
 43.0,
 30.0,
 26.0,
 28.0,
 18,
 20,
 22,
 46,
 25.0,
 22.0,
 33.0,
 20,
 40,
 25,
 24,
 17,
 16,
 10,
 44.0,
 10,
 19,
 21,
 44,
 28.0,
 23,
 58,
 26,
 21,
 21.0,
 23.0,
 34.0,
 4]

In [27]:
# Inserting found null values into dataframe
for x in range(len(test_nulls_ids)):
    index = test_df_dropped.loc[test_df_dropped['PassengerId'] == test_nulls_ids[x]].index
    test_df_dropped.loc[index, ['Age']] = test_ages_to_fill[x]

In [28]:
# Checking one last time for null values 
test_df_dropped.info()

test_df_dropped

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 13 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  418 non-null    int64  
 1   Survived     418 non-null    int64  
 2   Pclass       418 non-null    int64  
 3   last_name    418 non-null    object 
 4   first_name   418 non-null    object 
 5   Sex          418 non-null    object 
 6   Age          418 non-null    float64
 7   SibSp        418 non-null    int64  
 8   Parch        418 non-null    int64  
 9   Ticket       418 non-null    object 
 10  Fare         418 non-null    float64
 11  Embarked     418 non-null    object 
 12  Deck         418 non-null    object 
dtypes: float64(2), int64(5), object(6)
memory usage: 42.6+ KB


,PassengerId,Survived,Pclass,last_name,first_name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked,Deck
0,892,0,3,Kelly,Mr James,male,34.5,0,0,330911,7.8292,Q,"D,E,F,G"
1,893,1,3,Wilkes,Mrs James (Ellen Needs),female,47.0,1,0,363272,7.0000,S,"D,E,F,G"
2,894,0,2,Myles,Mr Thomas Francis,male,62.0,0,0,240276,9.6875,Q,"D,E,F,G"
3,895,0,3,Wirz,Mr Albert,male,27.0,0,0,315154,8.6625,S,"D,E,F,G"
4,896,1,3,Hirvonen,Mrs Alexander (Helga E Lindqvist),female,22.0,1,1,3101298,12.2875,S,"D,E,F,G"
...,...,...,...,...,...,...,...,...,...,...,...,...,...
413,1305,0,3,Spector,Mr Woolf,male,23.0,0,0,A.5. 3236,8.0500,S,"D,E,F,G"
414,1306,1,1,Oliva y Ocana,Dona Fermina,female,39.0,0,0,PC 17758,108.9000,C,"A,B,C,D,E"
415,1307,0,3,Saether,Mr Simon Sivertsen,male,38.5,0,0,SOTON/O.Q. 3101262,7.2500,S,"D,E,F,G"
416,1308,0,3,Ware,Mr Frederick,male,34.0,0,0,359309,8.0500,S,"D,E,F,G"


In [29]:
# Checking if age values were inserted correctly
test_inserted_ages = list(test_df_dropped.loc[test_df_dropped['PassengerId'].isin(test_nulls_ids)]['Age'])

if test_inserted_ages == test_ages_to_fill:
    print('Holy shit you did it')
    test_df_dropped.to_csv('../Resources/test_clean.csv', index=False)
else:
    print('Damn you suck')

Holy shit you did it


# Train Data

In [30]:
# Displaying train csv as a dataframe
train_df = pd.read_csv('../Resources/train.csv')

train_df.info()

train_df

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


In [31]:
# Splitting Name to first and last
train_df[['last_name', 'first_name']] = train_df['Name'].str.split(', ', expand=True)
# Removing period in first name to make matching easier later
train_df['first_name'] = train_df['first_name'].str.replace('.', '')
# Removing extra quotes some first names have
train_df['first_name'] = train_df['first_name'].str.replace('"', '')
# Reorganizing columns and dropping Cabin/Name column
train_df_dropped = train_df[['PassengerId', 'Survived', 'Pclass', 'last_name','first_name', 'Sex', 'Age', 'SibSp', 'Parch',
       'Ticket', 'Fare', 'Embarked']]

train_df_dropped

,PassengerId,Survived,Pclass,last_name,first_name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked
0,1,0,3,Braund,Mr Owen Harris,male,22.0,1,0,A/5 21171,7.2500,S
1,2,1,1,Cumings,Mrs John Bradley (Florence Briggs Thayer),female,38.0,1,0,PC 17599,71.2833,C
2,3,1,3,Heikkinen,Miss Laina,female,26.0,0,0,STON/O2. 3101282,7.9250,S
3,4,1,1,Futrelle,Mrs Jacques Heath (Lily May Peel),female,35.0,1,0,113803,53.1000,S
4,5,0,3,Allen,Mr William Henry,male,35.0,0,0,373450,8.0500,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,Montvila,Rev Juozas,male,27.0,0,0,211536,13.0000,S
887,888,1,1,Graham,Miss Margaret Edith,female,19.0,0,0,112053,30.0000,S
888,889,0,3,Johnston,Miss Catherine Helen Carrie,female,NaN,1,2,W./C. 6607,23.4500,S
889,890,1,1,Behr,Mr Karl Howell,male,26.0,0,0,111369,30.0000,C


In [32]:
# Creating Deck column, binning passengers by their class
deck_floors_train = []


for class_status in train_df_dropped['Pclass']:
    if class_status == 1:
        deck_floors_train.append('A,B,C,D,E')
    else:
        deck_floors_train.append('D,E,F,G')

i = 0

for k in range(len(train_df_dropped)):
    train_df_dropped.loc[k, ['Deck']] = deck_floors_train[i]
    i += 1


train_df_dropped

,PassengerId,Survived,Pclass,last_name,first_name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked,Deck
0,1,0,3,Braund,Mr Owen Harris,male,22.0,1,0,A/5 21171,7.2500,S,"D,E,F,G"
1,2,1,1,Cumings,Mrs John Bradley (Florence Briggs Thayer),female,38.0,1,0,PC 17599,71.2833,C,"A,B,C,D,E"
2,3,1,3,Heikkinen,Miss Laina,female,26.0,0,0,STON/O2. 3101282,7.9250,S,"D,E,F,G"
3,4,1,1,Futrelle,Mrs Jacques Heath (Lily May Peel),female,35.0,1,0,113803,53.1000,S,"A,B,C,D,E"
4,5,0,3,Allen,Mr William Henry,male,35.0,0,0,373450,8.0500,S,"D,E,F,G"
...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,Montvila,Rev Juozas,male,27.0,0,0,211536,13.0000,S,"D,E,F,G"
887,888,1,1,Graham,Miss Margaret Edith,female,19.0,0,0,112053,30.0000,S,"A,B,C,D,E"
888,889,0,3,Johnston,Miss Catherine Helen Carrie,female,NaN,1,2,W./C. 6607,23.4500,S,"D,E,F,G"
889,890,1,1,Behr,Mr Karl Howell,male,26.0,0,0,111369,30.0000,C,"A,B,C,D,E"


In [33]:
# Finding the 2 nulls in the 'Embarked' column and filling them in
embarked_nulls_df = train_df_dropped.loc[train_df_dropped['Embarked'].isna()]
embarked_nulls_index = embarked_nulls_df.index
embarked_nulls_ids = embarked_nulls_df['PassengerId'].values

for x in range(len(embarked_nulls_ids)):
    train_df_dropped.loc[embarked_nulls_index[x], ['Embarked']] = 'S'

display(embarked_nulls_df)
display(train_df_dropped.iloc[embarked_nulls_index])

,PassengerId,Survived,Pclass,last_name,first_name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked,Deck
61,62,1,1,Icard,Miss Amelie,female,38.0,0,0,113572,80.0,NaN,"A,B,C,D,E"
829,830,1,1,Stone,Mrs George Nelson (Martha Evelyn),female,62.0,0,0,113572,80.0,NaN,"A,B,C,D,E"


,PassengerId,Survived,Pclass,last_name,first_name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked,Deck
61,62,1,1,Icard,Miss Amelie,female,38.0,0,0,113572,80.0,S,"A,B,C,D,E"
829,830,1,1,Stone,Mrs George Nelson (Martha Evelyn),female,62.0,0,0,113572,80.0,S,"A,B,C,D,E"


In [34]:
# Need to check if dataset contains the 2 'NK' columns from full_table_df
for x in range(len(nk_last_names)):
    first_name = nk_first_names[x]
    last_name = nk_last_names[x]

    search = train_df_dropped.loc[train_df_dropped['last_name'] == last_name]
    refined_search = search.loc[search['first_name'] == first_name]

    index = refined_search.index

    if len(refined_search) == 1:
        train_df_dropped = train_df_dropped.drop(index=index)
        train_df_dropped = train_df_dropped.reset_index(drop=True)

In [35]:
# Checking age is only null column
train_df_dropped.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 889 entries, 0 to 888
Data columns (total 13 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  889 non-null    int64  
 1   Survived     889 non-null    int64  
 2   Pclass       889 non-null    int64  
 3   last_name    889 non-null    object 
 4   first_name   889 non-null    object 
 5   Sex          889 non-null    object 
 6   Age          714 non-null    float64
 7   SibSp        889 non-null    int64  
 8   Parch        889 non-null    int64  
 9   Ticket       889 non-null    object 
 10  Fare         889 non-null    float64
 11  Embarked     889 non-null    object 
 12  Deck         889 non-null    object 
dtypes: float64(2), int64(5), object(6)
memory usage: 90.4+ KB


In [36]:
# Creating age nulls only df and extracting last_name, first_name, and id 
train_nulls_only_df = train_df_dropped.loc[train_df_dropped['Age'].isna()]

train_nulls_last_names = train_nulls_only_df['last_name'].values
train_nulls_first_names = train_nulls_only_df['first_name'].values
train_nulls_ids = train_nulls_only_df['PassengerId'].values

train_nulls_only_df

,PassengerId,Survived,Pclass,last_name,first_name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked,Deck
5,6,0,3,Moran,Mr James,male,NaN,0,0,330877,8.4583,Q,"D,E,F,G"
17,18,1,2,Williams,Mr Charles Eugene,male,NaN,0,0,244373,13.0000,S,"D,E,F,G"
19,20,1,3,Masselmani,Mrs Fatima,female,NaN,0,0,2649,7.2250,C,"D,E,F,G"
26,27,0,3,Emir,Mr Farred Chehab,male,NaN,0,0,2631,7.2250,C,"D,E,F,G"
28,29,1,3,O'Dwyer,Miss Ellen Nellie,female,NaN,0,0,330959,7.8792,Q,"D,E,F,G"
...,...,...,...,...,...,...,...,...,...,...,...,...,...
857,860,0,3,Razi,Mr Raihed,male,NaN,0,0,2629,7.2292,C,"D,E,F,G"
861,864,0,3,Sage,Miss Dorothy Edith Dolly,female,NaN,8,2,CA. 2343,69.5500,S,"D,E,F,G"
866,869,0,3,van Melkebeke,Mr Philemon,male,NaN,0,0,345777,9.5000,S,"D,E,F,G"
876,879,0,3,Laleff,Mr Kristo,male,NaN,0,0,349217,7.8958,S,"D,E,F,G"


In [37]:
# Making sure data frame will all names are loaded
full_table_df_clean

,last_name,first_name,age
0,Allen,Miss Elisabeth Walton,29.0
1,Allison,Mr Hudson Joshua Creighton,30.0
2,Allison,Mrs Bessie Waldo,25.0
3,Allison,Miss Helen Loraine,2.0
4,Allison,Master Hudson Trevor,0.9
...,...,...,...
1310,Youssiff (Sam’Aan),Mr Gerios,45.0
1311,Zakarian,Mr Ortin,27.0
1312,Zakarian,Mr Mapriededer,22.0
1313,Zenni,Mr Philip,22.0


In [38]:
# Creating loop to find missing age values from full table to fill in for test csv
train_ages_to_fill = []
manually_fill_train_ages_empty = []

for index, last_name in enumerate(train_nulls_last_names):
    first_name = train_nulls_first_names[index]

    search = full_table_df_clean.loc[full_table_df_clean['last_name'] == last_name]   
    refined_search = search.loc[search['first_name'] == first_name]

    if len(refined_search) == 1:
        train_ages_to_fill.append(refined_search['age'].values[0])
    else:
        train_ages_to_fill.append(0)
        manually_fill_train_ages_empty.append({'last_name': last_name, 'first_name': first_name, 'age': ''})

In [39]:
# Printing results
print(train_ages_to_fill)
print(manually_fill_train_ages_empty)
print(f'We started with {len(train_ages_to_fill)} nulls')
print(f'Our search refined to manually go over {len(manually_fill_train_ages_empty)}')

[0, 23.0, 0, 0, 0, 23.0, 0, 19.0, 0, 29.0, 20.0, 0, 16.0, 45.0, 0, 0, 23.0, 28.0, 0, 20.0, 22.0, 0, 29.0, 0, 19.0, 19.0, 0, 0, 27.0, 37.0, 4.0, 0, 0, 0, 7.0, 0, 39.0, 0, 28.0, 0, 16.0, 22.0, 0, 0, 21.0, 0, 0, 19.0, 0, 0, 0, 0, 0, 0, 57.0, 0, 47.0, 0, 24.0, 0, 0, 0, 0, 29.0, 0, 30.0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 27.0, 18.0, 0, 42.0, 28.0, 21.0, 0, 0, 28.0, 0, 0, 28.0, 0, 0, 0, 34.0, 0, 22.0, 39.0, 40.0, 0, 0, 19.0, 0, 0, 0, 0, 66.0, 30.0, 0, 0, 0, 0, 0, 69.0, 0, 0, 0, 30.0, 40.0, 0, 0, 22.0, 0, 0, 22.0, 40.0, 31.0, 0, 42.0, 0, 0, 25.0, 22.0, 0, 29.0, 36.0, 32.0, 0, 23.0, 0, 27.0, 49.0, 0, 19.0, 0, 38.0, 0, 0, 41.0, 19.0, 0, 0, 20.0, 32.0, 45.0, 21.0, 45.0, 0, 29.0, 20.0, 0, 0, 0, 0, 42.0, 0, 42.0, 23.0, 19.0, 30.0, 0, 0, 18.0, 0, 30.0, 0, 0, 0, 0]
[{'last_name': 'Moran', 'first_name': 'Mr James', 'age': ''}, {'last_name': 'Masselmani', 'first_name': 'Mrs Fatima', 'age': ''}, {'last_name': 'Emir', 'first_name': 'Mr Farred Chehab', 'age': ''}, {'last_name': "O'Dwyer", 'first_name': 'Miss El

In [40]:
# Creating list to fill ages
manually_fill_train_ages = [
{'last_name': 'Moran', 'first_name': 'Mr James', 'age': 10},
 {'last_name': 'Masselmani', 'first_name': 'Mrs Fatima', 'age': 22},
 {'last_name': 'Emir', 'first_name': 'Mr Farred Chehab', 'age': 26},
 {'last_name': "O'Dwyer", 'first_name': 'Miss Ellen Nellie', 'age': 22},
 {'last_name': 'Spencer', 'first_name': 'Mrs William Augustus (Marie Eugenie)', 'age': 48},
 {'last_name': 'Mamee', 'first_name': 'Mr Hanna', 'age': 20},
 {'last_name': "O'Driscoll", 'first_name': 'Miss Bridget', 'age': 27},
 {'last_name': 'Stewart', 'first_name': 'Mr Albert A', 'age': 64},
 {'last_name': 'Moubarek', 'first_name': 'Master Gerios', 'age': 7},
 {'last_name': 'McDermott', 'first_name': 'Miss Brigdet Delia', 'age': 31},
 {'last_name': 'Petroff', 'first_name': 'Mr Pastcho (Pentcho)', 'age': 29},
 {'last_name': 'Moran', 'first_name': 'Miss Bertha', 'age': 28},
 {'last_name': 'Peter', 'first_name': 'Miss Anna', 'age': 2},
 {'last_name': 'Boulos', 'first_name': 'Mrs Joseph (Sultana)', 'age': 40},
 {'last_name': 'Chibnall', 'first_name': 'Mrs (Edith Martha Bowerman)', 'age': 48},
 {'last_name': 'Baumann', 'first_name': 'Mr John D', 'age': 48},
 {'last_name': 'Lefebre', 'first_name': 'Master Henry Forbes', 'age': 5},
 {'last_name': 'Pernot', 'first_name': 'Mr Rene', 'age': 39},
 {'last_name': "O'Brien", 'first_name': 'Mrs Thomas (Johanna Hannah Godfrey)', 'age': 27},
 {'last_name': 'Madigan', 'first_name': 'Miss Margaret Maggie', 'age': 21},
 {'last_name': 'Nenkoff', 'first_name': 'Mr Christo', 'age': 22},
 {'last_name': 'Lefebre', 'first_name': 'Miss Mathilde', 'age': 12},
 {'last_name': 'Zabour', 'first_name': 'Miss Thamine', 'age': 19},
 {'last_name': 'Murphy', 'first_name': 'Miss Katherine Kate', 'age': 18},
 {'last_name': 'Thorne', 'first_name': 'Mrs Gertrude Maybelle', 'age': 37},
 {'last_name': 'Smith', 'first_name': 'Mr Thomas', 'age': 26},
 {'last_name': 'Henry', 'first_name': 'Miss Delia', 'age': 22},
 {'last_name': 'Cairns', 'first_name': 'Mr Alexander', 'age': 28},
 {'last_name': 'Healy', 'first_name': 'Miss Hanora Nora', 'age': 33},
 {'last_name': 'Parkes', 'first_name': 'Mr Francis Frank', 'age': 21},
 {'last_name': 'Lewy', 'first_name': 'Mr Ervin G', 'age': 30},
 {'last_name': 'Kelly', 'first_name': 'Miss Anna Katherine Annie Kate', 'age': 20},
 {'last_name': 'Keane', 'first_name': 'Miss Nora A', 'age': 48},
 {'last_name': 'Williams', 'first_name': 'Mr Howard Hugh Harry', 'age': 28},
 {'last_name': 'Fleming', 'first_name': 'Miss Margaret', 'age': 42},
 {'last_name': 'Sage', 'first_name': 'Mr George John Jr', 'age': 30},
 {'last_name': 'Frauenthal', 'first_name': 'Mrs Henry William (Clara Heinsheimer)', 'age': 43},
 {'last_name': 'Davison', 'first_name': 'Mrs Thomas Henry (Mary E Finck)', 'age': 34},
 {'last_name': 'Williams-Lambert', 'first_name': 'Mr Fletcher Fellows', 'age': 44},
 {'last_name': 'Yousif', 'first_name': 'Mr Wazli', 'age': 23},
 {'last_name': 'McGovern', 'first_name': 'Miss Mary', 'age': 21},
 {'last_name': 'Mockler', 'first_name': 'Miss Helen Mary Ellie', 'age': 23},
 {'last_name': "O'Brien", 'first_name': 'Mr Thomas', 'age': 26},
 {'last_name': 'Moussa', 'first_name': 'Mrs (Mantoura Boulos)', 'age': 47},
 {'last_name': 'Jermyn', 'first_name': 'Miss Annie', 'age': 27},
 {'last_name': 'Meyer', 'first_name': 'Mrs Edgar Joseph (Leila Saks)', 'age': 25},
 {'last_name': 'Lefebre', 'first_name': 'Miss Ida', 'age': 3},
 {'last_name': 'Meek', 'first_name': 'Mrs Thomas (Annie Louise Rowley)', 'age': 32},
 {'last_name': 'Wiseman', 'first_name': 'Mr Phillippe', 'age': 53},
 {'last_name': 'Thorneycroft', 'first_name': 'Mrs Percival (Florence Kate White)', 'age': 32},
 {'last_name': 'Johannesen-Bratthammer', 'first_name': 'Mr Bernt', 'age': 29},
 {'last_name': 'Peduzzi', 'first_name': 'Mr Joseph', 'age': 24},
 {'last_name': 'Kenyon', 'first_name': 'Mrs Frederick R (Marion)', 'age': 40},
 {'last_name': "O'Connor", 'first_name': 'Mr Maurice', 'age': 15},
 {'last_name': 'Campbell', 'first_name': 'Mr William', 'age': 20},
 {'last_name': 'Frost', 'first_name': 'Mr Anthony Wood Archie', 'age': 38},
 {'last_name': 'Lefebre', 'first_name': 'Miss Jeannie', 'age': 8},
 {'last_name': 'Yousseff', 'first_name': 'Mr Gerious', 'age': 45},
 {'last_name': 'Shellard', 'first_name': 'Mr Frederick William', 'age': 55},
 {'last_name': "O'Sullivan", 'first_name': 'Miss Bridget Mary', 'age': 21},
 {'last_name': 'Bradley', 'first_name': 'Mr George (George Arthur Brayton)', 'age': 37},
 {'last_name': 'Lahoud', 'first_name': 'Mr Sarkis', 'age': 35},
 {'last_name': 'Kassem', 'first_name': 'Mr Fared', 'age': 19},
 {'last_name': 'Farthing', 'first_name': 'Mr John', 'age': 48},
 {'last_name': 'Toufik', 'first_name': 'Mr Nakli', 'age': 17},
 {'last_name': 'Peter', 'first_name': 'Mrs Catherine (Catherine Rizk)', 'age': 23},
 {'last_name': 'Padro y Manent', 'first_name': 'Mr Julian', 'age': 26},
 {'last_name': "O'Brien", 'first_name': 'Mr Timothy', 'age': 22},
 {'last_name': 'Robbins', 'first_name': 'Mr Victor', 'age': 45},
 {'last_name': 'Meanwell', 'first_name': 'Miss (Marion Ogden)', 'age': 62},
 {'last_name': 'Doharr', 'first_name': 'Mr Tannous', 'age': 28},
 {'last_name': 'Caram', 'first_name': 'Mrs Joseph (Maria Elias)', 'age': 28},
 {'last_name': 'Paulner', 'first_name': 'Mr Uscher', 'age': 16},
 {'last_name': 'Boulos', 'first_name': 'Mr Hanna', 'age': 18},
 {'last_name': 'Harrington', 'first_name': 'Mr Charles H', 'age': 37},
 {'last_name': 'Jardin', 'first_name': 'Mr Jose Neto', 'age': 21},
 {'last_name': "O'Connell", 'first_name': 'Mr Patrick D', 'age': 18},
 {'last_name': 'Willey', 'first_name': 'Mr Edward', 'age': 20},
 {'last_name': "O'Leary", 'first_name': 'Miss Hanora Norah', 'age': 16},
 {'last_name': 'Taylor', 'first_name': 'Mrs Elmer Zebley (Juliet Cummins Wright)', 'age': 49},
 {'last_name': 'Peters', 'first_name': 'Miss Katie', 'age': 27},
 {'last_name': 'Mullens', 'first_name': 'Miss Katherine Katie', 'age': 21},
 {'last_name': 'Moubarek', 'first_name': 'Master Halim Gonios (William George)', 'age': 4},
 {'last_name': 'Mannion', 'first_name': 'Miss Margareth', 'age': 28},
 {'last_name': 'Knight', 'first_name': 'Mr Robert J', 'age': 42},
 {'last_name': 'Moran', 'first_name': 'Mr Daniel J', 'age': 28},
 {'last_name': 'Kilgannon', 'first_name': 'Mr Thomas J', 'age': 21},
 {'last_name': 'Johnston', 'first_name': 'Mr Andrew G', 'age': 35},
 {'last_name': 'Keane', 'first_name': 'Mr Andrew Andy', 'age': 23},
 {'last_name': 'Sage', 'first_name': 'Miss Stella Anna', 'age': 20},
 {'last_name': 'Fry', 'first_name': 'Mr Richard', 'age': 39},
 {'last_name': 'Sirota', 'first_name': 'Mr Maurice', 'age': 19},
 {'last_name': 'Marechal', 'first_name': 'Mr Pierre', 'age': 29},
 {'last_name': 'Goldenberg', 'first_name': 'Mrs Samuel L (Edwiga Grabowska)', 'age': 46},
 {'last_name': 'Sage', 'first_name': 'Miss Dorothy Edith Dolly', 'age': 14},
 {'last_name': 'van Melkebeke', 'first_name': 'Mr Philemon', 'age': 23},
 {'last_name': 'Laleff', 'first_name': 'Mr Kristo', 'age': 23},
 {'last_name': 'Johnston', 'first_name': 'Miss Catherine Helen Carrie', 'age': 7}]

In [41]:
# Merging queried ages to make a complete list to fill test csv nulls
i = 0

for age in range(len(train_ages_to_fill)):
    if train_ages_to_fill[age] == 0:
       train_ages_to_fill[age] = manually_fill_train_ages[i]['age']
       i += 1

train_ages_to_fill

[10,
 23.0,
 22,
 26,
 22,
 23.0,
 48,
 19.0,
 20,
 29.0,
 20.0,
 27,
 16.0,
 45.0,
 64,
 7,
 23.0,
 28.0,
 31,
 20.0,
 22.0,
 29,
 29.0,
 28,
 19.0,
 19.0,
 2,
 40,
 27.0,
 37.0,
 4.0,
 48,
 48,
 5,
 7.0,
 39,
 39.0,
 27,
 28.0,
 21,
 16.0,
 22.0,
 22,
 12,
 21.0,
 19,
 18,
 19.0,
 37,
 26,
 22,
 28,
 33,
 21,
 57.0,
 30,
 47.0,
 20,
 24.0,
 48,
 28,
 42,
 30,
 29.0,
 43,
 30.0,
 34,
 44,
 23,
 21,
 23,
 26,
 47,
 27,
 25,
 27.0,
 18.0,
 3,
 42.0,
 28.0,
 21.0,
 32,
 53,
 28.0,
 32,
 29,
 28.0,
 24,
 40,
 15,
 34.0,
 20,
 22.0,
 39.0,
 40.0,
 38,
 8,
 19.0,
 45,
 55,
 21,
 37,
 66.0,
 30.0,
 35,
 19,
 48,
 17,
 23,
 69.0,
 26,
 22,
 45,
 30.0,
 40.0,
 62,
 28,
 22.0,
 28,
 16,
 22.0,
 40.0,
 31.0,
 18,
 42.0,
 37,
 21,
 25.0,
 22.0,
 18,
 29.0,
 36.0,
 32.0,
 20,
 23.0,
 16,
 27.0,
 49.0,
 49,
 19.0,
 27,
 38.0,
 21,
 4,
 41.0,
 19.0,
 28,
 42,
 20.0,
 32.0,
 45.0,
 21.0,
 45.0,
 28,
 29.0,
 20.0,
 21,
 35,
 23,
 20,
 42.0,
 39,
 42.0,
 23.0,
 19.0,
 30.0,
 19,
 29,
 18.0,
 46,
 30.0,

In [42]:
# Inserting found null values into dataframe
for x in range(len(train_nulls_ids)):
    index = train_df_dropped.loc[train_df_dropped['PassengerId'] == train_nulls_ids[x]].index
    train_df_dropped.loc[index, ['Age']] = train_ages_to_fill[x]

In [43]:
# Checking one last time for null values
train_df_dropped.info()

train_df_dropped

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 889 entries, 0 to 888
Data columns (total 13 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  889 non-null    int64  
 1   Survived     889 non-null    int64  
 2   Pclass       889 non-null    int64  
 3   last_name    889 non-null    object 
 4   first_name   889 non-null    object 
 5   Sex          889 non-null    object 
 6   Age          889 non-null    float64
 7   SibSp        889 non-null    int64  
 8   Parch        889 non-null    int64  
 9   Ticket       889 non-null    object 
 10  Fare         889 non-null    float64
 11  Embarked     889 non-null    object 
 12  Deck         889 non-null    object 
dtypes: float64(2), int64(5), object(6)
memory usage: 90.4+ KB


,PassengerId,Survived,Pclass,last_name,first_name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked,Deck
0,1,0,3,Braund,Mr Owen Harris,male,22.0,1,0,A/5 21171,7.2500,S,"D,E,F,G"
1,2,1,1,Cumings,Mrs John Bradley (Florence Briggs Thayer),female,38.0,1,0,PC 17599,71.2833,C,"A,B,C,D,E"
2,3,1,3,Heikkinen,Miss Laina,female,26.0,0,0,STON/O2. 3101282,7.9250,S,"D,E,F,G"
3,4,1,1,Futrelle,Mrs Jacques Heath (Lily May Peel),female,35.0,1,0,113803,53.1000,S,"A,B,C,D,E"
4,5,0,3,Allen,Mr William Henry,male,35.0,0,0,373450,8.0500,S,"D,E,F,G"
...,...,...,...,...,...,...,...,...,...,...,...,...,...
884,887,0,2,Montvila,Rev Juozas,male,27.0,0,0,211536,13.0000,S,"D,E,F,G"
885,888,1,1,Graham,Miss Margaret Edith,female,19.0,0,0,112053,30.0000,S,"A,B,C,D,E"
886,889,0,3,Johnston,Miss Catherine Helen Carrie,female,7.0,1,2,W./C. 6607,23.4500,S,"D,E,F,G"
887,890,1,1,Behr,Mr Karl Howell,male,26.0,0,0,111369,30.0000,C,"A,B,C,D,E"


In [44]:
# Checking if age values were inserted correctly
train_inserted_ages = list(train_df_dropped.loc[train_df_dropped['PassengerId'].isin(train_nulls_ids)]['Age'])

if train_inserted_ages == train_ages_to_fill:
    print('Holy shit you did it')
    train_df_dropped.to_csv('../Resources/train_clean.csv', index=False)
else:
    print('Damn you suck')

Holy shit you did it


# Merging Test CSV and Train CSV

In [45]:
# Merging test_clean and train_clean into one dataframe for Tableau
pd.concat(map(pd.read_csv, 
              ['../Resources/train_clean.csv', 
               '../Resources/test_clean.csv']), 
               ignore_index=True).to_csv('../Resources/merged_clean.csv', index=False)

In [51]:
pd.read_csv('../Resources/merged_clean.csv')

,PassengerId,Survived,Pclass,last_name,first_name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked,Deck
0,1,0,3,Braund,Mr Owen Harris,male,22.0,1,0,A/5 21171,7.2500,S,"D,E,F,G"
1,2,1,1,Cumings,Mrs John Bradley (Florence Briggs Thayer),female,38.0,1,0,PC 17599,71.2833,C,"A,B,C,D,E"
2,3,1,3,Heikkinen,Miss Laina,female,26.0,0,0,STON/O2. 3101282,7.9250,S,"D,E,F,G"
3,4,1,1,Futrelle,Mrs Jacques Heath (Lily May Peel),female,35.0,1,0,113803,53.1000,S,"A,B,C,D,E"
4,5,0,3,Allen,Mr William Henry,male,35.0,0,0,373450,8.0500,S,"D,E,F,G"
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1302,1305,0,3,Spector,Mr Woolf,male,23.0,0,0,A.5. 3236,8.0500,S,"D,E,F,G"
1303,1306,1,1,Oliva y Ocana,Dona Fermina,female,39.0,0,0,PC 17758,108.9000,C,"A,B,C,D,E"
1304,1307,0,3,Saether,Mr Simon Sivertsen,male,38.5,0,0,SOTON/O.Q. 3101262,7.2500,S,"D,E,F,G"
1305,1308,0,3,Ware,Mr Frederick,male,34.0,0,0,359309,8.0500,S,"D,E,F,G"


# MongoDB

In [103]:
# Writing train_clean df rows into dictionaries
with open('../Resources/train_clean.csv') as csvf:
    csvReader = csv.DictReader(csvf)

    dict_list = []

    for index, row in enumerate(csvReader):
        train_row_dict = {}

        train_dict = row
        dict_list.append(train_dict)

display(dict_list)

# Saving data df dictionary as a JSON file
with open('../Resources/train_clean.json', 'w') as openfile:
    json.dump(dict_list, openfile)

[{'PassengerId': '1',
  'Survived': '0',
  'Pclass': '3',
  'last_name': 'Braund',
  'first_name': 'Mr Owen Harris',
  'Sex': 'male',
  'Age': '22.0',
  'SibSp': '1',
  'Parch': '0',
  'Ticket': 'A/5 21171',
  'Fare': '7.25',
  'Embarked': 'S',
  'Deck': 'D,E,F,G'},
 {'PassengerId': '2',
  'Survived': '1',
  'Pclass': '1',
  'last_name': 'Cumings',
  'first_name': 'Mrs John Bradley (Florence Briggs Thayer)',
  'Sex': 'female',
  'Age': '38.0',
  'SibSp': '1',
  'Parch': '0',
  'Ticket': 'PC 17599',
  'Fare': '71.2833',
  'Embarked': 'C',
  'Deck': 'A,B,C,D,E'},
 {'PassengerId': '3',
  'Survived': '1',
  'Pclass': '3',
  'last_name': 'Heikkinen',
  'first_name': 'Miss Laina',
  'Sex': 'female',
  'Age': '26.0',
  'SibSp': '0',
  'Parch': '0',
  'Ticket': 'STON/O2. 3101282',
  'Fare': '7.925',
  'Embarked': 'S',
  'Deck': 'D,E,F,G'},
 {'PassengerId': '4',
  'Survived': '1',
  'Pclass': '1',
  'last_name': 'Futrelle',
  'first_name': 'Mrs Jacques Heath (Lily May Peel)',
  'Sex': 'female',

In [104]:
# Creating connection to client
conn  = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

client.list_database_names()

['admin', 'config', 'flu_vaccines_db', 'local', 'trips', 'uk_food']

In [105]:
# Drop any existing databases with the name titanic
client.drop_database('titanic')

In [106]:
# Creating a fresh database
db = client['titanic']

In [107]:
# Creating collections for each dataset
myCollection = db['train']

In [109]:
# Inserting data into database
myCollection.insert_many(dict_list)